In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, dropout):
        super(TemporalBlock, self).__init__()
        padding = (kernel_size - 1) * dilation  # Correct padding to maintain the output size
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.batchnorm1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.batchnorm2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        if self.downsample is not None:
            nn.init.xavier_uniform_(self.downsample.weight)

    def forward(self, x):
        # Applying convolutions
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        # Handling residual connection
        res = x if self.downsample is None else self.downsample(x)
        # Trimming output to match residual size if needed
        out = out[:, :, :x.size(2)]  # Ensure the output size matches the input size
        
        return F.relu(out + res)


In [3]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     dropout=dropout)]
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class IMTCNModel(nn.Module):
    def __init__(self, input_size, num_channels, output_size, kernel_size=2, dropout=0.2):
        super(IMTCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        y1 = self.tcn(x)  # Multi-scale temporal feature extraction
        y1 = y1[:, :, -1]  # Take the last output along the temporal dimension for classification
        out = self.linear(y1)
        return out


In [5]:
path=r'data'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.long).to(device)  # 确保标签是长整型

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
X_train = np.load(path + "/X_train.npy")
y_train = np.load(path + "/y_train.npy")
X_test = np.load(path + "/X_test.npy")
y_test = np.load(path + "/y_test.npy")
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
import torch.optim as optim
input_size = 7  # Number of features (e.g., parameters from QAR data)
num_channels = [16, 32, 64]  # Channels for each TCN block
output_size = 2  # For example, binary classification
model = IMTCNModel(input_size, num_channels, output_size)
class_weights = torch.tensor([1., 1.], device=device) 
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
import torch
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Number of rounds
num_rounds = 5
best_f1 = 0.0
best_model_path = 'IMTCN_best_model.pth'  # 最佳模型保存路径
# Initialize lists to store metrics for each round
accuracy_list = []
f1_list = []
roc_auc_list = []
mcc_list = []

for round_num in range(num_rounds):
    # Reinitialize the model and optimizer for each round
    model = IMTCNModel(input_size, num_channels, output_size).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 1000  # Set the number of epochs as needed
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)  # Move data and target to GPU
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target.long())  # Assuming target contains class indices for classification
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * data.size(0)
            probs = torch.sigmoid(outputs.squeeze(1))
            _, predicted = torch.max(probs, 1)
            all_labels.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Round {round_num + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        f1 = f1_score(all_labels, all_predictions)
        print(f1)
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with F1 score: {best_f1:.4f}')

    # 加载最好的模型
    model.load_state_dict(torch.load(best_model_path))
    print('Loaded best model for further evaluation or use.')

    # Evaluation loop
    model.eval()
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # Move data and target to GPU
            outputs = model(data)

            # Assuming binary classification; adjust for multi-class as needed
            probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Get probability of positive class
            preds = torch.argmax(outputs, dim=1)  # Get predicted class

            all_labels.extend(target.cpu().numpy())  # Move data back to CPU for metrics
            all_predictions.extend(preds.cpu().numpy())
            all_probs.extend(probabilities.cpu().numpy())

    # Calculate metrics for this round
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_predictions)

    # Store metrics
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    roc_auc_list.append(roc_auc)
    mcc_list.append(mcc)

    # Print metrics for this round
    print(f'Round {round_num + 1} Results:')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f'  ROC-AUC Score: {roc_auc:.4f}')
    print(f'  Matthews Correlation Coefficient (MCC): {mcc:.4f}')

# Calculate and print the average and standard deviation for each metric
print('\nFinal Results (Average ± Std Dev over 5 rounds):')
print(f'Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}')
print(f'F1 Score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}')
print(f'ROC-AUC Score: {np.mean(roc_auc_list):.4f} ± {np.std(roc_auc_list):.4f}')
print(f'Matthews Correlation Coefficient (MCC): {np.mean(mcc_list):.4f} ± {np.std(mcc_list):.4f}')


Round 1, Epoch 1/1000, Loss: 1.4164
0.3211822660098522
Saved best model with F1 score: 0.3212
Round 1, Epoch 2/1000, Loss: 0.7056
0.2846441947565543
Round 1, Epoch 3/1000, Loss: 0.8951
0.3622497616777884
Saved best model with F1 score: 0.3622
Round 1, Epoch 4/1000, Loss: 0.5944
0.4086845466155811
Saved best model with F1 score: 0.4087
Round 1, Epoch 5/1000, Loss: 0.6266
0.454954954954955
Saved best model with F1 score: 0.4550
Round 1, Epoch 6/1000, Loss: 0.5936
0.4597432905484248
Saved best model with F1 score: 0.4597
Round 1, Epoch 7/1000, Loss: 0.5964
0.4784580498866213
Saved best model with F1 score: 0.4785
Round 1, Epoch 8/1000, Loss: 0.7399
0.4064516129032258
Round 1, Epoch 9/1000, Loss: 0.7666
0.4318869828456105
Round 1, Epoch 10/1000, Loss: 0.7501
0.4476093591047813
Round 1, Epoch 11/1000, Loss: 0.6389
0.47956989247311826
Saved best model with F1 score: 0.4796
Round 1, Epoch 12/1000, Loss: 0.5631
0.5336273428886439
Saved best model with F1 score: 0.5336
Round 1, Epoch 13/1000, L

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TimeAwareAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim):
        super(TimeAwareAttention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attention_layer = nn.Linear(hidden_dim, attention_dim)
        self.context_vector = nn.Linear(attention_dim, 1, bias=False)

    def attention(self, lstm_output):
        attention_score = torch.tanh(self.attention_layer(lstm_output))
        attention_weights = F.softmax(self.context_vector(attention_score), dim=1)
        weighted_output = lstm_output * attention_weights
        return torch.sum(weighted_output, dim=1), attention_weights

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attention_out, attention_weights = self.attention(lstm_out)
        return attention_out, attention_weights

class SDTANModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim, output_dim):
        super(SDTANModel, self).__init__()
        self.time_attention = TimeAwareAttention(input_dim, hidden_dim, attention_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        attention_out, attention_weights = self.time_attention(x)
        output = self.fc(attention_out)
        return output, attention_weights

# Example usage
input_dim = 10  # Example input dimension (number of features)
hidden_dim = 64
attention_dim = 32
output_dim = 1  # Binary prediction: hard landing (1) or not (0)

model = SDTANModel(input_dim, hidden_dim, attention_dim, output_dim)
sample_input = torch.rand(16, 100, input_dim)  # (batch_size, sequence_length, input_dim)
output, attention_weights = model(sample_input)
print("Model output shape:", output.shape)


Model output shape: torch.Size([16, 1])


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TimeAwareAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim):
        super(TimeAwareAttention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attention_layer = nn.Linear(hidden_dim, attention_dim)
        self.context_vector = nn.Linear(attention_dim, 1, bias=False)

    def attention(self, lstm_output):
        attention_score = torch.tanh(self.attention_layer(lstm_output))
        attention_weights = F.softmax(self.context_vector(attention_score), dim=1)
        weighted_output = lstm_output * attention_weights
        return torch.sum(weighted_output, dim=1), attention_weights

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attention_out, attention_weights = self.attention(lstm_out)
        return attention_out, attention_weights

class SDTANClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim, num_classes):
        super(SDTANClassificationModel, self).__init__()
        self.time_attention = TimeAwareAttention(input_dim, hidden_dim, attention_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        attention_out, attention_weights = self.time_attention(x)
        output = self.fc(attention_out)
        output = F.softmax(output, dim=1)  # Applying softmax for multi-class classification
        return output, attention_weights

# Example usage
input_dim = 10  # Example input dimension (number of features)
hidden_dim = 64
attention_dim = 32
num_classes = 2  # Example: 3-class classification

model = SDTANClassificationModel(input_dim, hidden_dim, attention_dim, num_classes)
sample_input = torch.rand(16, 100, input_dim)  # (batch_size, sequence_length, input_dim)
output, attention_weights = model(sample_input)
print("Model output shape:", output.shape)


Model output shape: torch.Size([16, 2])


In [20]:
input_dim = 55  # Matching the last dimension of your data shape [32, 7, 55]
hidden_dim = 64
attention_dim = 32
num_classes = 2  # Example: 3-class classification, change according to your task

model = SDTANClassificationModel(input_dim, hidden_dim, attention_dim, num_classes)
sample_input = torch.rand(32, 7, 55)  # Matching the input data shape [32, 7, 55]
output, attention_weights = model(sample_input)
print("Model output shape:", output.shape)

Model output shape: torch.Size([32, 2])


In [35]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
import numpy as np

# Number of rounds
num_rounds = 5
best_f1 = 0.0
best_model_path = 'SDTAN_best_model.pth'  # 最佳模型保存路径
# Initialize lists to store metrics for each round
accuracy_list = []
f1_list = []
roc_auc_list = []
mcc_list = []

for round_num in range(num_rounds):
    # Reinitialize the model and optimizer for each round
    model = SDTANClassificationModel(input_dim, hidden_dim, attention_dim, num_classes).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 1000  # Set the number of epochs as needed
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs,_ = model(data)
            loss = criterion(outputs, target.long())  # Assuming target contains class indices for classification
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * data.size(0)
            probs = torch.sigmoid(outputs.squeeze(1))
            _, predicted = torch.max(probs, 1)
            all_labels.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Round {round_num + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        f1 = f1_score(all_labels, all_predictions)
        print(f1)
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with F1 score: {best_f1:.4f}')

    # 加载最好的模型
    model.load_state_dict(torch.load(best_model_path))
    print('Loaded best model for further evaluation or use.')

    # Evaluation loop
    model.eval()
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs,_ = model(data)

            # Assuming binary classification; adjust for multi-class as needed
            probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Get probability of positive class
            preds = torch.argmax(outputs, dim=1)  # Get predicted class

            all_labels.extend(target.cpu().numpy())
            all_predictions.extend(preds.cpu().numpy())
            all_probs.extend(probabilities.cpu().numpy())

    # Calculate metrics for this round
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_predictions)

    # Store metrics
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    roc_auc_list.append(roc_auc)
    mcc_list.append(mcc)

    # Print metrics for this round
    print(f'Round {round_num + 1} Results:')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f'  ROC-AUC Score: {roc_auc:.4f}')
    print(f'  Matthews Correlation Coefficient (MCC): {mcc:.4f}')

# Calculate and print the average and standard deviation for each metric
print('\nFinal Results (Average ± Std Dev over 5 rounds):')
print(f'Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}')
print(f'F1 Score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}')
print(f'ROC-AUC Score: {np.mean(roc_auc_list):.4f} ± {np.std(roc_auc_list):.4f}')
print(f'Matthews Correlation Coefficient (MCC): {np.mean(mcc_list):.4f} ± {np.std(mcc_list):.4f}')


Round 1, Epoch 1/1000, Loss: 0.6318
0.0
Round 1, Epoch 2/1000, Loss: 0.6264
0.0
Round 1, Epoch 3/1000, Loss: 0.6236
0.0
Round 1, Epoch 4/1000, Loss: 0.6170
0.05639097744360901
Saved best model with F1 score: 0.0564
Round 1, Epoch 5/1000, Loss: 0.6001
0.15194346289752653
Saved best model with F1 score: 0.1519
Round 1, Epoch 6/1000, Loss: 0.5771
0.32926829268292684
Saved best model with F1 score: 0.3293
Round 1, Epoch 7/1000, Loss: 0.5676
0.39999999999999997
Saved best model with F1 score: 0.4000
Round 1, Epoch 8/1000, Loss: 0.5605
0.4438280166435506
Saved best model with F1 score: 0.4438
Round 1, Epoch 9/1000, Loss: 0.5562
0.47556142668428003
Saved best model with F1 score: 0.4756
Round 1, Epoch 10/1000, Loss: 0.5456
0.5057766367137355
Saved best model with F1 score: 0.5058
Round 1, Epoch 11/1000, Loss: 0.5452
0.5090439276485789
Saved best model with F1 score: 0.5090
Round 1, Epoch 12/1000, Loss: 0.5404
0.5146496815286624
Saved best model with F1 score: 0.5146
Round 1, Epoch 13/1000, Lo

In [36]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
import numpy as np

# Number of rounds
num_rounds = 5

# Initialize lists to store metrics for each round
accuracy_list = []
f1_list = []
roc_auc_list = []
mcc_list = []

for round_num in range(num_rounds):
    # Reinitialize the model and optimizer for each round
    model = SDTANClassificationModel(input_dim, hidden_dim, attention_dim, num_classes).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 600  # Set the number of epochs as needed
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs,_ = model(data)
            loss = criterion(outputs, target.long())  # Assuming target contains class indices for classification
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * data.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Round {round_num + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # Evaluation loop
    model.eval()
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs,_ = model(data)

            # Assuming binary classification; adjust for multi-class as needed
            probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Get probability of positive class
            preds = torch.argmax(outputs, dim=1)  # Get predicted class

            all_labels.extend(target.cpu().numpy())
            all_predictions.extend(preds.cpu().numpy())
            all_probs.extend(probabilities.cpu().numpy())

    # Calculate metrics for this round
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_predictions)

    # Store metrics
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    roc_auc_list.append(roc_auc)
    mcc_list.append(mcc)

    # Print metrics for this round
    print(f'Round {round_num + 1} Results:')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f'  ROC-AUC Score: {roc_auc:.4f}')
    print(f'  Matthews Correlation Coefficient (MCC): {mcc:.4f}')

# Calculate and print the average and standard deviation for each metric
print('\nFinal Results (Average ± Std Dev over 5 rounds):')
print(f'Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}')
print(f'F1 Score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}')
print(f'ROC-AUC Score: {np.mean(roc_auc_list):.4f} ± {np.std(roc_auc_list):.4f}')
print(f'Matthews Correlation Coefficient (MCC): {np.mean(mcc_list):.4f} ± {np.std(mcc_list):.4f}')


Round 1, Epoch 1/600, Loss: 0.6310
Round 1, Epoch 2/600, Loss: 0.6257
Round 1, Epoch 3/600, Loss: 0.6183
Round 1, Epoch 4/600, Loss: 0.6029
Round 1, Epoch 5/600, Loss: 0.5818
Round 1, Epoch 6/600, Loss: 0.5655
Round 1, Epoch 7/600, Loss: 0.5639
Round 1, Epoch 8/600, Loss: 0.5575
Round 1, Epoch 9/600, Loss: 0.5510
Round 1, Epoch 10/600, Loss: 0.5496
Round 1, Epoch 11/600, Loss: 0.5468
Round 1, Epoch 12/600, Loss: 0.5388
Round 1, Epoch 13/600, Loss: 0.5457
Round 1, Epoch 14/600, Loss: 0.5425
Round 1, Epoch 15/600, Loss: 0.5325
Round 1, Epoch 16/600, Loss: 0.5368
Round 1, Epoch 17/600, Loss: 0.5390
Round 1, Epoch 18/600, Loss: 0.5390
Round 1, Epoch 19/600, Loss: 0.5425
Round 1, Epoch 20/600, Loss: 0.5282
Round 1, Epoch 21/600, Loss: 0.5268
Round 1, Epoch 22/600, Loss: 0.5302
Round 1, Epoch 23/600, Loss: 0.5339
Round 1, Epoch 24/600, Loss: 0.5232
Round 1, Epoch 25/600, Loss: 0.5193
Round 1, Epoch 26/600, Loss: 0.5258
Round 1, Epoch 27/600, Loss: 0.5321
Round 1, Epoch 28/600, Loss: 0.5213
R